<a href="https://colab.research.google.com/github/Alejandro-RL/RNA-Atividade-1.2/blob/main/RNA_Processamento_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento de Dados

# Imports e o arquivo em sí

No total, o arquivo possui 243984 registros.

In [ ]:
import pandas as pd
!pip install geopandas
import geopandas as gpd
import datetime

In [ ]:
!wget https://www.dropbox.com/s/ys5r54e6a5je0y0/Manaus.csv

In [ ]:
csv = pd.read_csv("/content/Manaus.csv", sep=';')

# Visão Geral dos Casos Confirmados
Levando em conta somente casos confirmados, que totalizam 80420.

In [ ]:
df = csv[csv['_classificacao'] == 'Confirmado']

Quantos atributos descrevem cada exemplo? Quais são eles?
(Feito usando o .columns para extrair as colunas, depois o .unique() para ver os valores)
R = 
* Datas de Evolução, Notifição e Sintomas da Doença 
* Classificação (Descartado, Confirmado, Em análise) 
* Conclusão (Recuperado, Óbito, Óbito por outras causas)
* Origem (SIVEP, GAL, eSUS VE) [???]
* Comorbidades, cada uma com uma coluna (Diabetes, Hemato, Hepatica, Imuno, Neurologica, Obessidade, Renal, Respiratoria)
* Critério (Laboratorial, Clínico, Clínico Epidemiológico, Clínico-Imagem)
* Etnia (Etnias Indígenas)
* Evolução (Internado, Internado em UTI, Em tratamento domiciliar)
* Idade
* Faixa Etária (< 1, 1-4, 5-9, 10-14, 15-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80 ou +)
* Gestante
* Profissional de Saúde
* Raça ([Parda, Branca, Preta, Amarela, Indigena, Ignorado)
* Se Notifição (valores numéricos inteiros) [???]
* Sexo (M, F, I)
* Sintomas, cada um com uma coluna (Dispneia, Febre, Garganta,Tosse, Outros)
* SRAG (Síndrome Respiratória Aguda, os valores são os vírus que causam isso, incluindo a covid-19) [???]
* Testes, cada um com uma coluna (Anticorpo, Antígeno, PCR)
* Localização, cada um com uma coluna (Bairro, Bairro_Mapa, Distrito)
* Taxa (valores numéricos fracionados) [???]

In [61]:
df.columns

Index(['_dt_evolucao', '_dt_notificacao', '_dt_sintomas', '_classificacao',
       '_conclusao', '_origem', '_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria', '_criterio', '_etnia',
       '_evolução', '_idade', '_faixa etária', '_gestante', '_profiss_saude',
       '_raca', '_se_notificacao', '_sexo', '_sintoma_dispneia',
       '_sintoma_febre', '_sintoma_garganta', '_sintoma_tosse',
       '_sintoma_outros', '_srag', '_teste_anticorpo', '_teste_antigeno',
       '_teste_pcr', '_bairro', '_bairro_mapa', '_distrito', '_taxa'],
      dtype='object')



---



Quantos casos confirmados há em Manaus, cumulativamente?

R = 80420 casos

In [62]:
len(df)

80420



---



A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o
mais recente? Leve em conta a data de notificação.

R:
*   Registro mais antigo: 16 de Fevereiro, 2020
*   Registro mais recente: 29 de Dezembro, 2020

In [212]:
#Extraindo as datas
datas = [i for i in df['_dt_notificacao']]
#Retirando a hora já que são todas iguais
datas = [str(i).strip("00:00") for i in datas]
#datas agora uma lista onde cada elemento é do tipo [dd/mm/aaaa]
datas = [i.split('/') for i in datas]

#Transformando os valores da lista datas no tipo datetime
#Colocando os resultados na lista datas1
datas1 = []
for i in datas:
  #Filtrando as datas vazias
  if("nan" not in i):
    ano = int(i[2])
    mes = int(i[1])
    dia = int(i[0])
    datas1.append(datetime.datetime(ano, mes, dia)) 

#Agora basta usar o sort
datas1 = sorted(datas1)
print("Em mm/dd/aa")
print("Primeira data no registro: "+datas1[0].strftime("%x"))
print("Última data no registro: "+datas1[-1].strftime("%x"))

Em mm/dd/aa
Primeira data no registro: 02/16/20
Última data no registro: 12/29/20




---



In [ ]:
bairros = df['_bairro'].unique()
bairros = [str(i).lower() for i in bairros]
bairros.sort()